In [ ]:
import pandas as pd
import sqlite3

In [ ]:
df_bal = pd.read_csv('./data/balance.csv', delimiter=';', dtype='str')
df_2015 = pd.read_csv('./data/2015.csv', dtype='str')
df_2015 = df_2015[df_2015['TRANSACTION ID'].notna()]
df_2016 = pd.read_csv('./data/2016.csv', dtype='str')
df_2016 = df_2016[df_2016['TRANSACTION ID'].notna()]
df_2017 = pd.read_csv('./data/2017.csv', dtype='str')
df_2017 = df_2017[df_2017['TRANSACTION ID'].notna()]

In [ ]:
df_trans = pd.concat([df_2015, df_2016, df_2017])

In [ ]:
df_trans.columns = [i.strip().lower() for i in df_trans.columns]

In [ ]:
df_bal.columns = [i.strip().lower() for i in df_bal.columns]

In [ ]:
df_comb = pd.merge(df_trans, df_bal, how='left', on='date')

In [ ]:
df_comb.to_csv('./data/combined_transactions.csv')

In [ ]:
df_comb['words'] = df_comb['description'].apply(lambda x: x.split())

In [ ]:
def sort_categories(desc_words):
    if 'Call' in desc_words or 'Put' in desc_words:
        return 'option'
    elif 'Bought' in desc_words or 'Sold' in desc_words:
        return 'stock'
    elif 'EXPIRATION' in desc_words or 'EXERCISE' in desc_words:
        return 'expiration'
    elif 'ASSIGNMENT' in desc_words:
        return 'assignment'
    elif 'DIVIDEND' in desc_words:
        return 'dividend'
    elif 'FUNDING' in desc_words:
        return 'funds'
    elif 'MARGIN' in desc_words:
        return 'margin interest'
#     elif 'OFF-CYCLE' in desc_words:
#         return 'off-cycle interest'
    elif 'FUTURES' in desc_words:
        return 'futures'
#     elif 'MARK' in desc_words:
#         return 'mark to market'
#     elif 'REBATE' in desc_words:
#         return 'rebate'
#     elif 'INTERNAL' in desc_words and 'CASH' in desc_words:
#         return 'internal cash transfer'
    elif 'INTEREST' in desc_words and 'ADJUSTMENT' in desc_words:
        return 'interest adjustment'
#     elif 'INTERNAL' in desc_words and 'ACCOUNT' in desc_words:
#         return 'internal account transfer'
#     elif 'MISCELLANEOUS' in desc_words:
#         return 'miscellaneous'
    else:
        return 'miscellaneous'
    
def get_underlying(symbol):
    try:
        underlying = symbol.split()[0]
        return underlying
    except BaseException:
        return 

In [ ]:
df_comb['category'] = df_comb['words'].apply(lambda x: sort_categories(x))
df_comb['underlying'] = df_comb['symbol'].apply(lambda x: get_underlying(x))

In [ ]:
df_comb

In [ ]:
df_comb.columns

In [ ]:
df_comb.to_excel('./data/processed.xlsx', index=False)

In [ ]:
df_comb[df_comb['category']=='stocks']

In [ ]:
df_comb[((df_comb['category']=='stock') | (df_comb['category']=='option')) & (df_comb['underlying']=='NFLX')]

In [ ]:
df_comb[['price', 'commission']].fillna(0)